In [1]:
import os
import re
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow import keras
from sklearn.model_selection import train_test_split

### 데이터 읽기

In [2]:
path = r"./data/fruits-images-dataset-object-detection/Train File/Train File"

train_images, test_images, train_labels, test_labels = [], [], [], []

for file in sorted(os.listdir(path)):
   if file.endswith(".jpg") or file.endswith("jpeg"):
       img = cv2.cvtColor(cv2.resize(cv2.imread(os.path.join(path, file)), (128, 128)), cv2.COLOR_BGR2RGB)
       train_images.append(img)
       if "Hog Pulm" in file:
           file = file.replace("Hog Pulm", "HogPlum")
       elif "HogPulm" in file:
           file = file.replace("HogPulm", "HogPlum")
       elif "Lichi" in file:
           file = file.replace("Lichi", "Litchi")
       train_labels.append(re.findall("[a-zA-Z]+", file)[0])

In [3]:
path = r"./data/fruits-images-dataset-object-detection/Test File/Test File"

for file in sorted(os.listdir(path)):
   if file.endswith(".jpg") or file.endswith(".jpeg"):
       img = cv2.cvtColor(cv2.resize(cv2.imread(os.path.join(path, file)), (128, 128)), cv2.COLOR_BGR2RGB)
       test_images.append(img)
       test_labels.append(re.findall("[a-zA-Z]+", file)[0])

In [4]:
train_images = np.array(train_images)
test_images = np.array(test_images)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [5]:
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

(320, 128, 128, 3) (320,) (80, 128, 128, 3) (80,)


In [6]:
class_names = np.unique(train_labels)

class_map = {i : class_name for i, class_name in enumerate(class_names)}
reverse_class_map = {val : key for key, val in class_map.items()}

print(class_map)

{0: 'Apple', 1: 'Banana', 2: 'Grapes', 3: 'Guava', 4: 'HogPlum', 5: 'Jackfruit', 6: 'Litchi', 7: 'Mango', 8: 'Orange', 9: 'Papaya'}


### 라벨을 숫자로 변환

In [7]:
train_list = []

for label in train_labels:
    key = reverse_class_map[label]
    train_list.append(key)

In [8]:
y_train = np.array(train_list)

In [9]:
test_list = []

for label in test_labels:
    key = reverse_class_map[label]
    test_list.append(key)

In [10]:
y_test = np.array(test_list)

### 스케일링 적용

In [11]:
scaled_train = train_images / 255
scaled_test = test_images / 255

In [12]:
scaled_sub, scaled_val, y_sub, y_val = train_test_split(scaled_train, y_train, test_size = 0.2, stratify = y_train, random_state = 7)

In [13]:
scaled_sub.shape, scaled_val.shape, scaled_test.shape

((256, 128, 128, 3), (64, 128, 128, 3), (80, 128, 128, 3))

### 모델 구성

In [14]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(64, kernel_size = 8, activation = "relu", padding = "same", input_shape = (128, 128, 3)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(64, kernel_size = 8, activation = "relu", padding = "same"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Conv2D(128, kernel_size = 4, activation = "relu", padding = "same"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(128, kernel_size = 4, activation = "relu", padding = "same"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation = "relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10, activation = "softmax"))

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
adam = keras.optimizers.Adam(learning_rate = 0.0005)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 4, restore_best_weights = True)

In [16]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer = adam, metrics = ["accuracy"])

### 모델 학습 및 평가

In [17]:
model.fit(scaled_sub, y_sub, epochs = 25, validation_data = (scaled_val, y_val), callbacks = [early_stopping_cb], batch_size = 8)

Epoch 1/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.1429 - loss: 2.8412 - val_accuracy: 0.1719 - val_loss: 2.7074
Epoch 2/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.3464 - loss: 1.9620 - val_accuracy: 0.1250 - val_loss: 4.0083
Epoch 3/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.3313 - loss: 1.9723 - val_accuracy: 0.1406 - val_loss: 5.2835
Epoch 4/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.3252 - loss: 2.1224 - val_accuracy: 0.0781 - val_loss: 26.9647
Epoch 5/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.2237 - loss: 2.5059 - val_accuracy: 0.0781 - val_loss: 6.1189


In [18]:
model.evaluate(scaled_test, y_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.1227 - loss: 2.8455   


[2.8750243186950684, 0.11249999701976776]